In [41]:
import pymysql as mysql

server = "sql7.freemysqlhosting.net"
user = "sql7271447"
password = "hi7DSvMiPz"
database = "sql7271447"

Для храниения информации о том, что смотрит каждый пользователь, создана отдельная таблица FUConnection. Где хранится id пользователя, id фильма, прогресс, статус и оценка.

Изначально прогресс равен 0, а оценка не указана вообще.

Давайте попробуем записать что-то в эту таблицу. Для этого возьмем 5 рандомных аниме и 1 рандомного пользователя.

In [35]:
def look_something(table, thing):
    db = mysql.connect(server, user, password, database)
    with db.cursor() as cursor:
        sql = "SELECT {} FROM {}"
        cursor.execute(sql.format(thing, table))
        items = cursor.fetchall()
    db.commit()
    db.close()
    return items

def look_something_one(table, thing):
    db = mysql.connect(server, user, password, database)
    with db.cursor() as cursor:
        sql = "SELECT {} FROM {}"
        cursor.execute(sql.format(thing, table))
        items = cursor.fetchone()
    db.commit()
    db.close()
    return items

def add(table, columns, values):
    db = mysql.connect(server,
                       user,
                       password,
                       database)
    with db.cursor() as cursor:
        sql = "INSERT INTO {}({}) VALUES ({})"
        cursor.execute(sql.format(table,
                                  ", ".join(columns),
                                  "'" + str("', '".join(values)) + "'"))
    db.commit()
    db.close()

def look_something_random(table, thing, num):
    db = mysql.connect(server, user, password, database)
    with db.cursor() as cursor:
        sql = '''
        SELECT {} FROM {}
        ORDER BY RAND()
        LIMIT {}'''
        cursor.execute(sql.format(thing, table, num))
        items = cursor.fetchall()
    db.commit()
    db.close()
    return items


In [42]:
mal_user = look_something_one('Users', '*')
print(mal_user)
mal_user = look_something_one('Users', 'id')
print(mal_user)

(1, 'karthiga', 'karthiga@pochta.ru', datetime.date(2013, 3, 3), 'Female', 'Chennai, India ', '')
(1,)


In [43]:
films = look_something_random('Films', 'id', 10)
for r in films:
    print(r)

(88,)
(74,)
(77,)
(22,)
(93,)
(68,)
(52,)
(18,)
(60,)
(31,)


Давайте добавим эти десять фильмов в список просмотров выбранного пользователя

In [79]:
for film in films:
    add('TUConnection',
        ['User_id', 'Film_id'],
        [str(mal_user[0]), str(film[0])])

Теперь посмотрим на результат 

In [80]:
look_something_random('TUConnection', '*', 5)

((24, 1, 22, 0, None, 'Watching'),
 (29, 1, 60, 0, None, 'Watching'),
 (28, 1, 18, 0, None, 'Watching'),
 (21, 1, 88, 0, None, 'Watching'),
 (30, 1, 31, 0, None, 'Watching'))

Мы видим, что у нас есть id пользователя, а также id тайтлов, которые он смотрит. Также у нас есть количество эпизодов, оценка, которая временно отсутствует, а также статус просмотра. Давайте обпределим статусы просмотра и напишем функцию, которая будет выводить на экран пользователя, название тайтла, который он смотрит, а также количество просмотренных эпизодов.

In [71]:
def look_tuc(user_id):
    db = mysql.connect(server, user, password, database)
    with db.cursor() as cursor:
        query = '''
        SELECT O.Episodes, O.Score, O.Status , U.Username, F.Title 
      FROM TUConnection O 
      JOIN Users U ON O.User_id = U.id 
      JOIN Films F ON O.Film_id = F.id
      WHERE O.User_id = {}'''
        cursor.execute(query.format(user_id))
        items = cursor.fetchall()
    db.commit()
    db.close()
    return items

Мы точно знаем, что у нас есть юзер с id 1, давайте попробуем посмотреть, что у нас получится...

In [81]:
look_tuc(1)

((0, None, 'Watching', 'karthiga', 'Higurashi no Naku Koro ni'),
 (0, None, 'Watching', 'karthiga', 'Sore ga Seiyuu!'),
 (0, None, 'Watching', 'karthiga', 'Sidonia no Kishi'),
 (0, None, 'Watching', 'karthiga', 'Non Non Biyori Repeat'),
 (0, None, 'Watching', 'karthiga', 'Mobile Suit Gundam'),
 (0, None, 'Watching', 'karthiga', 'Fullmetal Alchemist: Brotherhood'),
 (0,
  None,
  'Watching',
  'karthiga',
  'Watashi ga Motenai no wa Dou Kangaetemo Omaera ga Warui!'),
 (0, None, 'Watching', 'karthiga', 'Fairy Tail'),
 (0, None, 'Watching', 'karthiga', 'Gantz'),
 (0, None, 'Watching', 'karthiga', 'D-Frag!'))

Теперь красиво это выведем на экран

In [73]:
def show_film_list(user_id):
    for item in look_tuc(user_id):
        print('''
        User:\t{}
        Title:\t{}
        Status:\t{}
        Score:\t{}
        Watched {} epesodes'''.format(item[3], item[4], item[2], item[1], item[0]))

In [82]:
show_film_list(1)


        User:	karthiga
        Title:	Higurashi no Naku Koro ni
        Status:	Watching
        Score:	None
        Watched 0 epesodes

        User:	karthiga
        Title:	Sore ga Seiyuu!
        Status:	Watching
        Score:	None
        Watched 0 epesodes

        User:	karthiga
        Title:	Sidonia no Kishi
        Status:	Watching
        Score:	None
        Watched 0 epesodes

        User:	karthiga
        Title:	Non Non Biyori Repeat
        Status:	Watching
        Score:	None
        Watched 0 epesodes

        User:	karthiga
        Title:	Mobile Suit Gundam
        Status:	Watching
        Score:	None
        Watched 0 epesodes

        User:	karthiga
        Title:	Fullmetal Alchemist: Brotherhood
        Status:	Watching
        Score:	None
        Watched 0 epesodes

        User:	karthiga
        Title:	Watashi ga Motenai no wa Dou Kangaetemo Omaera ga Warui!
        Status:	Watching
        Score:	None
        Watched 0 epesodes

        User:	karthiga
        T

Теперь давайте создадим список возможных статусов тайтлов и напишим функции, которые будут менять количество просмотренных эпизодов, статус и оцеку

In [58]:
conf_status = ['Watching', 'Completed', 'On-Hold', 'Dropped', 'Plan to Watch']

def get_max_ep(title_id):
    db = mysql.connect(server, user, password, database)
    with db.cursor() as cursor:
        sql = "SELECT Episodes FROM Films WHERE id = {}"
        cursor.execute(sql.format(title_id))
        items = cursor.fetchone()
    db.commit()
    db.close()
    return items

In [66]:
def change_data(user_id, film_id, episodes=None, status=None, score=None):
    max_ep = get_max_ep(film_id)[0]
    db = mysql.connect(server, user, password, database)
    to_change = []
    if episodes and type(episodes) == int and episodes in range(0, max_ep):
        to_change.append('Episodes = {}'.format(str(episodes)))
    if status and status in conf_status:
        to_change.append('Status = "{}"'.format(str(status)))
    if score and score in range(0, 10):
        to_change.append('Score = {}'.format(str(score)))
    if to_change:
        with db.cursor() as cursor:
            query = '''
            UPDATE TUConnection
            SET {}
            WHERE User_id = {}
            AND Film_id = {}'''
            cursor.execute(query.format(', '.join(to_change), user_id, film_id))
    db.commit()
    db.close()

In [67]:
change_data(1, 52, episodes=10, score=4)

Не сломалось и хорошо. Давайте попробуем поменять информацию для всех тайтлов для этого человека, а потом выведем данные еще раз.

In [83]:
import random

In [86]:
for film in films:
    change_data(1,
                film[0],
                episodes=random.choice(range(0, 12)),
                score=random.choice(range(0, 10)),
                status=random.choice(conf_status))

In [87]:
show_film_list(1)


        User:	karthiga
        Title:	Higurashi no Naku Koro ni
        Status:	Plan to Watch
        Score:	1
        Watched 9 epesodes

        User:	karthiga
        Title:	Sore ga Seiyuu!
        Status:	Plan to Watch
        Score:	3
        Watched 7 epesodes

        User:	karthiga
        Title:	Sidonia no Kishi
        Status:	Plan to Watch
        Score:	7
        Watched 8 epesodes

        User:	karthiga
        Title:	Non Non Biyori Repeat
        Status:	Completed
        Score:	5
        Watched 5 epesodes

        User:	karthiga
        Title:	Mobile Suit Gundam
        Status:	Completed
        Score:	None
        Watched 6 epesodes

        User:	karthiga
        Title:	Fullmetal Alchemist: Brotherhood
        Status:	Plan to Watch
        Score:	None
        Watched 11 epesodes

        User:	karthiga
        Title:	Watashi ga Motenai no wa Dou Kangaetemo Omaera ga Warui!
        Status:	Plan to Watch
        Score:	9
        Watched 9 epesodes

        User:	karth

Возможно и не очень логично, но все работает. Теперь мы можем менять, добавлять и смотреть. Предположим пользователь решил удалить какой-то тайтл из своего списка. Надо сделать.

In [88]:
def delete_films(user_id, film_id):
    db = mysql.connect(server, user, password, database)
    with db.cursor() as cursor:
        sql = "DELETE FROM TUConnection WHERE User_id = {} AND Film_id = {}"
        cursor.execute(sql.format(user_id, film_id))
    db.commit()
    db.close()

In [89]:
delete_films(1, 22)

In [90]:
show_film_list(1)


        User:	karthiga
        Title:	Higurashi no Naku Koro ni
        Status:	Plan to Watch
        Score:	1
        Watched 9 epesodes

        User:	karthiga
        Title:	Sore ga Seiyuu!
        Status:	Plan to Watch
        Score:	3
        Watched 7 epesodes

        User:	karthiga
        Title:	Sidonia no Kishi
        Status:	Plan to Watch
        Score:	7
        Watched 8 epesodes

        User:	karthiga
        Title:	Mobile Suit Gundam
        Status:	Completed
        Score:	None
        Watched 6 epesodes

        User:	karthiga
        Title:	Fullmetal Alchemist: Brotherhood
        Status:	Plan to Watch
        Score:	None
        Watched 11 epesodes

        User:	karthiga
        Title:	Watashi ga Motenai no wa Dou Kangaetemo Omaera ga Warui!
        Status:	Plan to Watch
        Score:	9
        Watched 9 epesodes

        User:	karthiga
        Title:	Fairy Tail
        Status:	Plan to Watch
        Score:	4
        Watched 5 epesodes

        User:	karthiga
   

Non Non Biyori Repeat больше нет... Добавление ревью можно осуществлять при помощи функции **add**, поэтому тут не очень интересно. Давайте добавим функцию, которая будет выводить среднюю оценку по всем аниме в списке. Для этого добавим для уже существующего списка фильмов, оценки двух любых пользователей.

In [91]:
for film in films:
    add('TUConnection',
        ['User_id', 'Film_id'],
        [str(2), str(film[0])])

In [92]:
for film in films:
    change_data(2,
                film[0],
                episodes=random.choice(range(0, 12)),
                score=random.choice(range(0, 10)),
                status=random.choice(conf_status))

In [93]:
for film in films:
    add('TUConnection',
        ['User_id', 'Film_id'],
        [str(3), str(film[0])])

In [94]:
for film in films:
    change_data(3,
                film[0],
                episodes=random.choice(range(0, 12)),
                score=random.choice(range(0, 10)),
                status=random.choice(conf_status))

In [95]:
def show_mean_score(film_id):
    db = mysql.connect(server, user, password, database)
    with db.cursor() as cursor:
        query = '''
        SELECT AVG(O.Score), F.Title 
      FROM TUConnection O 
      JOIN Films F ON O.Film_id = F.id
      WHERE O.Film_id = {}'''
        cursor.execute(query.format(film_id))
        items = cursor.fetchall()
    db.commit()
    db.close()
    return items

In [99]:
show_mean_score(31)

((Decimal('3.3333'), 'D-Frag!'),)

In [100]:
for film_id in films:
    for score, name in show_mean_score(film_id[0]):
        print('''
        Title:\t{}
        Score:\t{}'''.format(name, float(score)))


        Title:	Higurashi no Naku Koro ni
        Score:	5.3333

        Title:	Sore ga Seiyuu!
        Score:	3.6667

        Title:	Sidonia no Kishi
        Score:	4.6667

        Title:	Non Non Biyori Repeat
        Score:	4.0

        Title:	Mobile Suit Gundam
        Score:	4.5

        Title:	Fullmetal Alchemist: Brotherhood
        Score:	9.0

        Title:	Watashi ga Motenai no wa Dou Kangaetemo Omaera ga Warui!
        Score:	6.3333

        Title:	Fairy Tail
        Score:	5.0

        Title:	Gantz
        Score:	6.6667

        Title:	D-Frag!
        Score:	3.3333


Ура, мы можем выводеть средние оценки пользователей для тайтлов, теперь можно показывать эти оценки и пользователи могут на них ориентироваться. 

Еще, будет неплохо, если при показе ревью, если пользователь оценил тайтл, то пускай при показе ревью, показывалась и оценка.

Добавим 2 ревью, один от пользователя, который оценил тайтл, и от второго, который не оценивал 


In [104]:
text1 = """
It is true that school-life is one of the most common genres in Japanese animation.
From specialty school (magic, cooking etc) to school-life with mysteries or supernatural.
In short, it is a genre that has been widely exploited in all respects.
As for Tsuki ga Kirei, it does not pretend to revolutionize the genre and does not propose an original plot at first."""

text2 = """When attempting to define romance, a few notions come to mind: a feeling of excitement, a remoteness from the mundaneness of everyday life, or perhaps a quality of mystery.
These are the qualities that litter the romance genre today.
However, one cannot discount the impact that simpler techniques create"""

add('Reviews', ['User_id', 'Film_id', 'Text'], [str(2), str(60), text1])
add('Reviews', ['User_id', 'Film_id', 'Text'], [str(6), str(60), text2])

In [121]:
look_something('Reviews', '*')

((1,
  1,
  60,
  '\nIt is true that school-life is one of the most common genres in Japanese animation.\nFrom specialty school (magic, cooking etc) to school-life with mysteries or supernatural.\nIn short, it is a genre that has been widely exploited in all respects.\nAs for Tsuki ga Kirei, it does not pretend to revolutionize the genre and does not propose an original plot at first.'),
 (2,
  1,
  60,
  'When attempting to define romance, a few notions come to mind: a feeling of excitement, a remoteness from the mundaneness of everyday life, or perhaps a quality of mystery.\nThese are the qualities that litter the romance genre today.\nHowever, one cannot discount the impact that simpler techniques create'),
 (3,
  2,
  60,
  '\nIt is true that school-life is one of the most common genres in Japanese animation.\nFrom specialty school (magic, cooking etc) to school-life with mysteries or supernatural.\nIn short, it is a genre that has been widely exploited in all respects.\nAs for Tsu

In [106]:
add('TUConnection',
        ['User_id', 'Film_id'],
        [str(6), str(60)])

In [208]:
def score_plus_review(film_id):
    db = mysql.connect(server, user, password, database)
    with db.cursor() as cursor:
        query = '''
        SELECT TUC.Score, R.Text, F.Title, U.Username
      FROM Reviews R
      JOIN TUConnection TUC ON R.Film_id = TUC.Film_id 
      JOIN Films F ON R.Film_id = F.id 
      JOIN Users U ON R.User_id = U.id
      WHERE R.Film_id = {}
      AND TUC.User_id = R.User_id'''
        cursor.execute(query.format(str(film_id)))
        items = cursor.fetchall()
        
    db.commit()
    db.close()
    return items

def show_reviews(title_id):
    for item in score_plus_review(title_id):
        if item[0] != None:
            print('''
Title:\t{}
User:\t{}
Score:\t{}
Text:\t{}'''.format(item[2], item[3], str(item[0]), item[1]))
        else:
            print('''
Title:\t{}
User:\t{}
Text:\t{}'''.format(item[2], item[3], item[1]))

In [209]:
show_reviews(60)


Title:	Gantz
User:	karthiga
Score:	9
Text:	
It is true that school-life is one of the most common genres in Japanese animation.
From specialty school (magic, cooking etc) to school-life with mysteries or supernatural.
In short, it is a genre that has been widely exploited in all respects.
As for Tsuki ga Kirei, it does not pretend to revolutionize the genre and does not propose an original plot at first.

Title:	Gantz
User:	karthiga
Score:	9
Text:	When attempting to define romance, a few notions come to mind: a feeling of excitement, a remoteness from the mundaneness of everyday life, or perhaps a quality of mystery.
These are the qualities that litter the romance genre today.
However, one cannot discount the impact that simpler techniques create

Title:	Gantz
User:	RedvelvetDaisuki
Score:	2
Text:	
It is true that school-life is one of the most common genres in Japanese animation.
From specialty school (magic, cooking etc) to school-life with mysteries or supernatural.
In short, it is